In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# ERA5 Dataset

In [2]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
from neural_lam import constants

import os

RAW_ERA5_PATH = constants.ERA5UKConstants.RAW_ERA5_PATH

In [3]:
val_metrics = constants.ERA5UKConstants.VAR_LEADS_METRICS_WATCH
val_metric_names = [constants.ERA5UKConstants.PARAM_NAMES[i] for i in val_metrics.keys()]
val_metric_names

['z500', 't850', 'q700', 'u850', 'v850', 'w850']

## Verify nwp_xy.npy and time step files

In [4]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/2022*.nc')
nc_files.sort()
nc_file = nc_files[0]
print(nc_file)

/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc


In [5]:
# dataset = "era5_uk"
# subset = era5_data_proc.uk_subset

# dataset = "era5_uk_big"
dataset = "era5_uk_big_coarse"
subset = era5_data_proc.uk_big_subset

data = xr.open_dataset(nc_file)
data = subset(data)
time = data['time'].values[2]
# time = np.datetime64('2022-06-01T00:00:00')
print(time)
sample = data.sel(time=time)
sample

2022-01-01T12:00:00.000000000


<xarray.Dataset> Size: 3MB
Dimensions:    (level: 8, latitude: 81, longitude: 81)
Coordinates:
  * longitude  (longitude) float32 324B 347.0 347.2 347.5 347.8 ... 6.5 6.75 7.0
  * latitude   (latitude) float32 324B 65.0 64.75 64.5 64.25 ... 45.5 45.25 45.0
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
    time       datetime64[ns] 8B 2022-01-01T12:00:00
Data variables:
    z          (level, latitude, longitude) float64 420kB 1.93e+05 ... 2.069e+03
    q          (level, latitude, longitude) float64 420kB 3.179e-06 ... 0.004964
    t          (level, latitude, longitude) float64 420kB 201.4 201.5 ... 297.0
    u          (level, latitude, longitude) float64 420kB 29.75 29.83 ... 1.054
    v          (level, latitude, longitude) float64 420kB 12.92 12.8 ... -0.2492
    w          (level, latitude, longitude) float64 420kB 0.006608 ... 0.4246
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [6]:
# Verify Data Variable Shapes
sample["z"].shape

(8, 81, 81)

In [7]:
# load time step data
sample_dir_path = f"data/{dataset}/samples/train"
sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
sample_file = sample_files[2]
print(sample_file)

time_step_data = np.load(sample_file)
print(time_step_data.shape)
time_step_data

data/era5_uk_big_coarse/samples/train/20220101120000.npy
(729, 48)


array([[ 1.92961335e+05,  1.28589659e+05,  9.83679591e+04, ...,
        -5.38659383e-01, -7.77642869e-01, -2.44279342e-01],
       [ 1.93019151e+05,  1.28650687e+05,  9.84257751e+04, ...,
        -6.91114366e-01, -2.74953468e-01, -1.81148938e-02],
       [ 1.93076967e+05,  1.28711715e+05,  9.84868031e+04, ...,
        -7.03017758e-01, -2.56640557e-01, -1.75605927e-01],
       ...,
       [ 2.00939941e+05,  1.35119654e+05,  1.04445062e+05, ...,
         2.03422189e-02, -1.35775346e-01, -4.42107917e-02],
       [ 2.00936729e+05,  1.35090746e+05,  1.04435426e+05, ...,
         6.33775592e-02,  2.02930807e-03,  9.81229516e-03],
       [ 2.00930305e+05,  1.35065050e+05,  1.04425790e+05, ...,
         2.62530464e-01,  4.36503117e-01,  4.51611268e-01]])

In [8]:
# load nwp_xy.npy
static_dir_path = f"data/{dataset}/static"
grid_xy = torch.tensor(
    np.load(os.path.join(static_dir_path, "nwp_xy.npy"))
)  # (2, N_y, N_x)
print(grid_xy.shape)
grid_xy = grid_xy.reshape(2, -1).T # (N_y * N_x, 2)
print(grid_xy.shape)


torch.Size([2, 27, 27])
torch.Size([729, 2])


In [9]:
# choose random lon/lat point and verify grid features match
test_point = 201
# grid_xy stores lat/lon in -180/180 format
# convert to 0/360 format
test_xy = grid_xy[test_point] % 360
print(test_xy)
time_step_data[test_point]

tensor([356.0000,  59.7500])


array([ 1.96089822e+05,  1.30600371e+05,  9.88979390e+04,  6.81687429e+04,
        5.26130310e+04,  3.92928704e+04,  1.24084374e+04, -7.73607180e+02,
        3.17917909e-06,  3.55128608e-06,  6.90024897e-06,  1.34905053e-04,
        3.47750249e-04,  9.54656746e-04,  4.98234278e-03,  6.97423148e-03,
        2.06069555e+02,  2.12819320e+02,  2.20114354e+02,  2.36724393e+02,
        2.49118928e+02,  2.59592630e+02,  2.77192002e+02,  2.85730413e+02,
        3.17542888e+01,  1.98428653e+01,  1.72581134e+00,  6.01676441e+00,
        9.41456557e+00,  8.55145853e+00,  1.43091473e+01,  8.27196722e-01,
        1.17152710e+01,  1.88339890e+01,  3.61979098e+01,  2.76046657e+01,
        2.55488949e+01,  2.30403652e+01,  2.47167973e+01,  1.41687355e+01,
        3.86059915e-03, -9.27400053e-02,  6.79557869e-02, -3.21651390e-01,
       -1.81099800e-01, -1.21582840e-01,  2.26313327e-02, -5.75367902e-03])

In [10]:
# find the sample from xarray
sample.sel(longitude=test_xy[0], latitude=test_xy[1])

<xarray.Dataset> Size: 432B
Dimensions:    (level: 8)
Coordinates:
    longitude  float32 4B 356.0
    latitude   float32 4B 59.75
  * level      (level) int32 32B 50 150 250 400 500 600 850 1000
    time       datetime64[ns] 8B 2022-01-01T12:00:00
Data variables:
    z          (level) float64 64B 1.961e+05 1.306e+05 ... 1.241e+04 -773.6
    q          (level) float64 64B 3.179e-06 3.551e-06 ... 0.004982 0.006974
    t          (level) float64 64B 206.1 212.8 220.1 236.7 ... 259.6 277.2 285.7
    u          (level) float64 64B 31.75 19.84 1.726 ... 8.551 14.31 0.8272
    v          (level) float64 64B 11.72 18.83 36.2 27.6 25.55 23.04 24.72 14.17
    w          (level) float64 64B 0.003861 -0.09274 ... 0.02263 -0.005754
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-26 11:59:48 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

## ERA5 Dataset Names

In [11]:
sample.data_vars

Data variables:
    z        (level, latitude, longitude) float64 420kB 1.93e+05 ... 2.069e+03
    q        (level, latitude, longitude) float64 420kB 3.179e-06 ... 0.004964
    t        (level, latitude, longitude) float64 420kB 201.4 201.5 ... 297.0
    u        (level, latitude, longitude) float64 420kB 29.75 29.83 ... 1.054
    v        (level, latitude, longitude) float64 420kB 12.92 12.8 ... -0.2492
    w        (level, latitude, longitude) float64 420kB 0.006608 ... 0.4246

In [12]:
list(sample.data_vars.keys())

['z', 'q', 't', 'u', 'v', 'w']

In [13]:
sample.level.values

array([  50,  150,  250,  400,  500,  600,  850, 1000], dtype=int32)

In [14]:
from neural_lam.constants import ERA5UKConstants

PARAM_SYMBOLS = list(sample.data_vars.keys())
LEVELS = list(sample.level.values)
PARAM_NAMES = [
    p + str(l)
    for p in PARAM_SYMBOLS
    for l in LEVELS
]

PARAM_UNITS = [
    sample[p].attrs["units"]
    for p in PARAM_SYMBOLS
    for _ in LEVELS
]

print(PARAM_NAMES)
print(PARAM_UNITS)

['z50', 'z150', 'z250', 'z400', 'z500', 'z600', 'z850', 'z1000', 'q50', 'q150', 'q250', 'q400', 'q500', 'q600', 'q850', 'q1000', 't50', 't150', 't250', 't400', 't500', 't600', 't850', 't1000', 'u50', 'u150', 'u250', 'u400', 'u500', 'u600', 'u850', 'u1000', 'v50', 'v150', 'v250', 'v400', 'v500', 'v600', 'v850', 'v1000', 'w50', 'w150', 'w250', 'w400', 'w500', 'w600', 'w850', 'w1000']
['m**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1']


## Grid Features

In [15]:
import torch

### Simple Grid Features

In [19]:
grid_features_simple_path = f"./data/{dataset}/static/grid_features_simple.pt"
grid_features = torch.load(grid_features_simple_path)
grid_features.shape

torch.Size([729, 3])

### Grid Features with Static Variables

In [23]:
grid_features_path = f"./data/{dataset}/static/grid_features.pt"
grid_features = torch.load(grid_features_path)
grid_features.shape

torch.Size([729, 5])

## Verify static variables

In [24]:
dataset_path = f"{RAW_ERA5_PATH}/static_variables.nc"
data = xr.open_dataset(dataset_path)
data

<xarray.Dataset> Size: 266MB
Dimensions:    (longitude: 1440, latitude: 721, time: 16)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 128B 2022-03-11T09:00:00 ... 2023-10-28T...
Data variables:
    z          (time, latitude, longitude) float64 133MB ...
    lsm        (time, latitude, longitude) float64 133MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-15 15:41:55 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [25]:
# surface geopotential should be time invariant but we find that it is not.
# values changed sometime in 2022 October
# 99.7% of the values are the same however

sample1 = data.sel(time=data['time'].values[3])["z"].values
sample2 = data.sel(time=data['time'].values[4])["z"].values
np.isclose(sample1, sample2).sum() / sample1.size

0.9972260748959778

### Select a spatial location from xarray and .npy to test

In [26]:
# choose random lon/lat point and verify grid features match
test_point = 100
test_xy = grid_xy[test_point] % 360
print(test_xy)

data.sel(time=data['time'].values[0], longitude=test_xy[0], latitude=test_xy[1]).to_array()

tensor([ 1.2500, 62.7500])


<xarray.DataArray (variable: 2)> Size: 16B
array([4.41451046, 0.        ])
Coordinates:
    longitude  float32 4B 1.25
    latitude   float32 4B 62.75
    time       datetime64[ns] 8B 2022-03-11T09:00:00
  * variable   (variable) object 16B 'z' 'lsm'
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-15 15:41:55 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [27]:
grid_features[test_point]

tensor([0.4579, 0.0218, 0.9998, 4.4145, 0.0000])

## Verify Grid Features Code

In [ ]:
static_dir_path = "data/era5_uk/static"

In [ ]:
grid_xy = torch.tensor(
    np.load(os.path.join(static_dir_path, "nwp_xy.npy"))
)  # (2, N_x, N_y)
print(grid_xy.shape)
grid_xy = grid_xy.reshape(2, -1).T # (N_x * N_y, 2)
print(grid_xy.shape)
grid_xy

torch.Size([2, 57, 65])
torch.Size([3705, 2])


tensor([[-10.0000,  63.0000],
        [-10.0000,  62.7500],
        [-10.0000,  62.5000],
        ...,
        [  4.0000,  47.5000],
        [  4.0000,  47.2500],
        [  4.0000,  47.0000]])

In [ ]:
grid_xy = np.radians(grid_xy)
grid_xy

tensor([[-0.1745,  1.0996],
        [-0.1745,  1.0952],
        [-0.1745,  1.0908],
        ...,
        [ 0.0698,  0.8290],
        [ 0.0698,  0.8247],
        [ 0.0698,  0.8203]])

In [ ]:
grid_lons = grid_xy[:, 0]
grid_lats = grid_xy[:, 1]
grid_features = torch.stack(
    (
        np.cos(grid_lats), 
        np.sin(grid_lons), 
        np.cos(grid_lons)
    ), 
    dim=1
)
print(grid_features.shape)

torch.Size([3705, 3])


In [ ]:
# PyTorch and NumPy
import torch
batch_size = 4
N_grid = 250
d_features = 42

prev_prev_state = torch.rand((batch_size, N_grid, d_features))
prev_state = torch.rand((batch_size, N_grid, d_features))
# sample_len = 5
target_states = [torch.rand((N_grid, d_features)), torch.rand((N_grid, d_features)), torch.rand((N_grid, d_features))]
target_states = torch.stack(target_states, dim=0)
target_states = target_states.unsqueeze(0).repeat(batch_size, 1, 1, 1)

forcings = torch.zeros(batch_size, target_states.shape[1], target_states.shape[2], 0) # (batch_size, sample_len-2, N_grid, d_forcing)
forcing = forcings[:, 2]
print(prev_prev_state.shape)
print(prev_state.shape)
print(target_states.shape)
print(forcings.shape)
print(forcing.shape)
grid_features = torch.cat(
    (
        prev_prev_state,
        prev_state,
        forcing
    ),
    dim=-1
)
grid_features.shape